# Forza 7 Telemetry Reciever
[William Kane](github.com/tubestubes)  
Python 3.9.4 - 19/09/21

> ### Sources
> - https://github.com/croozer2000/Forza-UDP-Data-Pythonhttps://github.com/croozer2000/Forza-UDP-Data-Python
> - https://forums.forzamotorsport.net/turn10_postst128499p8_Forza-Motorsport-7--Data-Out--feature-details.aspx?=https://forums.forzamotorsport.net/turn10_postst128499p8_Forza-Motorsport-7--Data-Out--feature-details.aspx?=


In [2]:
# From the standard library
import socket
import struct
import logging; logging.basicConfig(level=logging.WARNING)
import time

#!pip install jupyterplot
import jupyterplot
import pandas as pd

In [3]:
# Set the same port here and in the game
PORT = 6969 

In [4]:
# Tools to convert raw data to numbers
# The format of each of the 85 parameters.
formats = ['s32','u32','f32','f32','f32','f32','f32','f32','f32','f32','f32','f32','f32','f32','f32','f32','f32',
'f32','f32','f32','f32','f32','f32','f32','f32','f32','f32','f32','f32','s32','s32','s32','s32','f32','f32','f32',
'f32','f32','f32','f32','f32','f32','f32','f32','f32','f32','f32','f32','f32','f32','f32','f32','f32','s32','s32',
's32','s32','s32','f32','f32','f32','f32','f32','f32','f32','f32','f32','f32','f32','f32','f32','f32','f32','f32',
'f32','u16','u8','u8','u8','u8','u8','u8','s8','s8','s8'
]

keys = [ 'On/Off', 'Time', 'MaxRPM', 'IdleRPM', 'RPM', 'AccX', 'AccY', 'AccZ', 'VelX', 'VelY', 'VelZ', 'RotX', 
'RotY', 'RotZ', 'Yaw', 'Pitch', 'Roll', 'NSusFL', 'NSusFR', 'NSusRL', 'NSusRR', 'SlipFL', 'SlipFR', 'SlipRL','SlipRR', 
'WSpeedFL', 'WSpeedFR', 'WSpeedRL', 'WSpeedRR', 'OnRumFL', 'OnRumFR', 'OnRumRL', 'OnRumRR', 'PuddleFL', 'PuddleFR', 
'PuddleRL', 'PuddleRR', 'sRumFL', 'sRumFR', 'sRumRL', 'sRumRR', 'SAngFL', 'SAngFR', 'SAngRL', 'SAngRR', 'CSlipFL', 
'CSlipFR','CSlipRL','CSlipRR', 'SusFL', 'SusFR', 'SusRL', 'SusRR', 'Car', 'Class', 'PI', 'DriveTrain', 'NCylinders', 
'X', 'Y', 'Z', 'Speed', 'Power', 'Torque', 'TempFL', 'TempFR', 'TempRL', 'TempRR', 'Boost', 'Fuel', 'Dist', 'BestLap',
'LastLap', 'CurrentLap', 'RaceTime', 'Lap', 'Pos', 'Acc', 'Brake', 'Clutch', 'Handbreak', 'Gear', 'Steer', 'NLine', 'AiBreak'
]
logging.debug(dict(zip(keys, formats)))

def convert(raw):

    raw_i = 0  # Working index in the raw data
    output = {}

    for i, fmt in enumerate(formats):

        key = keys[i]
        logging.debug((i,fmt, key, raw_i))

        if fmt == 'f32':
            output[key] = struct.unpack('<f', raw[raw_i : raw_i + 4])[0]
            raw_i = raw_i + 4 

        if fmt == 'u32':
            output[key] = struct.unpack('<L',raw[raw_i : raw_i + 4])[0]
            raw_i = raw_i + 4 

        if fmt == 'u16':
            output[key] = struct.unpack('<H',raw[raw_i : raw_i + 2])[0]
            raw_i = raw_i + 2 
        
        if fmt == 's32':
            output[key] = struct.unpack('<i',raw[raw_i:raw_i + 4])[0]
            raw_i = raw_i + 4 
        
        if fmt == 'u8':
            output[key] = struct.unpack('<B',raw[raw_i:raw_i + 1])[0]
            raw_i = raw_i + 1 

        if fmt == 's8':
            output[key] = struct.unpack('<B',raw[raw_i:raw_i + 1])[0]
            raw_i = raw_i + 1 

    return output

In [ ]:
# Connect to UDP and receive packet
def receive(port:int ):
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as soc:
        soc.bind(('', port))
        raw = soc.recv(port)
        logging.info('Received data')
    logging.debug(raw)

    data = convert(raw)
    return data

def receive_one(port:int, param:str ):
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as soc:
        soc.bind(('', port))
        raw = soc.recv(port)
        logging.info('Received data')
    logging.debug(raw)

    data = convert(raw)
    return data[param]

In [ ]:
# Record one paramater live, with graph
def record_one(port, param, length):
    points = []
    plot = jupyterplot.ProgressPlot()
    start = time.time()
    while time.time() < start + length:
        data = receive(port)
        points.append( data[param])
        plot.update(data[param])
    plot.finalize()
    return points

def record_list(port, params: list, length):
    points = []
    plot = jupyterplot.ProgressPlot(line_names=params)
    start = time.time()
    while time.time() < start + length:
        data = receive(port)
        points.append( [data[param] for param in params] )
        plot.update( [[data[param] for param in params]] )
    plot.finalize()
    return points


In [ ]:
# Live demo
data = record_list(PORT, ['Speed', 'Acc','Brake', 'Gear', 'Lap'], 60*3)

In [ ]:
data

In [ ]:
df = pd.DataFrame(data, columns=['Speed', 'Acc','Brake', 'Gear', 'Lap'])
lap0 = df[df['Lap']==0]
lap0['Gear'] = lap0['Gear']*10
lap0.plot()

In [7]:
pd.DataFrame(zip(keys, formats)).to_csv('key.csv')